# 2.A NLP2SBVR

Initial version.

In [ ]:
from openai import OpenAI
import instructor

from typing import List
from pydantic import BaseModel, Field

import datetime
import os

import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
print(os.getenv("OPENAI_API_KEY"))

In [ ]:
LLM_MODEL = "gpt-4o-2024-08-06" #"gpt-4o"
EMBEDDING_MODEL ="text-embedding-3-small"
TEMPERATURE = 0

## Prompt engeneering

In [ ]:
system_prompt = """
Using the following taxonomy and templates, transform the following text into a JSON object according to the provided structure:

Output format:
{
  "input_text": "<original input text>",

  "transformed_rules": [
    {
      "rule_type": "Definitional Rule",
      "transformation": "<transformed rule statement>",
      "template_format": "T6"
    },
    {
      "rule_type": "Party Restriction Rule",
      "transformation": "<transformed rule statement>",
      "template_format": "T53"
    },
    {
      "rule_type": "Data Consistency Rule",
      "transformation": "<transformed rule statement>",
      "template_format": "T35"
    }
  ],

  "note": {
    "explanation": "The provided input was classified into three types of business rules following the book's taxonomy: Definitional Rule (explaining what total assets are), Party Restriction Rule (regarding control and asset limits), and Data Consistency Rule (setting a minimum asset threshold). These transformations align the text with clear business rule statements.",

    "templates_used": [
      {
        "template_name": "Definitional Rule Template",
        "template_format": "T6. {A|An|The|} <definitional rule statement subject> {<qualifying clause>|} <verb phrase> by definition <definition>."
      },
      {
        "template_name": "Party Restriction Rule Template",
        "template_format": "T53. A <party signifier 1> {<qualifying clause> | } may <predicate 1> only if {the <attribute signifier> of | } that <party signifier 1> <predicate 2>."
      },
      {
        "template_name": "Data Consistency Rule Template",
        "template_format": "T35. The <data item term> specified in each <transaction signifier> must be {<equality operator>} <literal>."
      }
    ]
  }
}

Your task is to transform the input text into business rules using the given templates and formats.

"""

In [ ]:
reference_document = """
§ 275.0-7 Small entities under the Investment Advisers Act for purposes of the Regulatory Flexibility Act.
(a) For purposes of Commission rulemaking in accordance with the provisions of Chapter Six of the Administrative Procedure Act (5 U.S.C. 601 et seq.) and unless otherwise defined for purposes of a particular rulemaking proceeding, the term small business or small organization for purposes of the Investment Advisers Act of 1940 shall mean an investment adviser that:
  (1) Has assets under management, as defined under Section 203A(a)(3) of the Act (15 U.S.C. 80b-3a(a)(2)) and reported on its annual updating amendment to Form ADV (17 CFR 279.1), of less than $25 million, or such higher amount as the Commission may by rule deem appropriate under Section 203A(a)(1)(A) of the Act (15 U.S.C. 80b-3a(a)(1)(A));
  (2) Did not have total assets of $5 million or more on the last day of the most recent fiscal year; and
  (3) Does not control, is not controlled by, and is not under common control with another investment adviser that has assets under management of $25 million or more (or such higher amount as the Commission may deem appropriate), or any person (other than a natural person) that had total assets of $5 million or more on the last day of the most recent fiscal year.
(b) For purposes of this section:
  (1) Control  means the power, directly or indirectly, to direct the management or policies of a person, whether through ownership of securities, by contract, or otherwise.
    (i) A person is presumed to control a corporation if the person:
      (A) Directly or indirectly has the right to vote 25 percent or more of a class of the corporation's voting securities; or
      (B) Has the power to sell or direct the sale of 25 percent or more of a class of the corporation's voting securities.
    (ii) A person is presumed to control a partnership if the person has the right to receive upon dissolution, or has contributed, 25 percent or more of the capital of the partnership.
    (iii) A person is presumed to control a limited liability company (LLC) if the person:
      (A) Directly or indirectly has the right to vote 25 percent or more of a class of the interests of the LLC;
      (B) Has the right to receive upon dissolution, or has contributed, 25 percent or more of the capital of the LLC; or
      (C) Is an elected manager of the LLC.
    (iv) A person is presumed to control a trust if the person is a trustee or managing agent of the trust.
  (2) Total assets  means the total assets as shown on the balance sheet of the investment adviser or other person described above under paragraph (a)(3) of this section, or the balance sheet of the investment adviser or such other person with its subsidiaries consolidated, whichever is larger.
"""

In [ ]:
term = "total assets"

input_text = "the total assets as shown on the balance sheet of the investment adviser or other person described above under paragraph (a)(3) of this section, or the balance sheet of the investment adviser or such other person with its subsidiaries consolidated, whichever is larger."

In [ ]:
user_prompt = f"""

# References:
{reference_document}

# Input text:
{term} means: {input_text}
"""

print(user_prompt)

## Transform rules to SBVR

In [ ]:
def query_llm(user_prompt: str, system_prompt: str, response_model: object) -> list:
    client = instructor.from_openai(OpenAI())
    resp = client.chat.completions.create(
        model=LLM_MODEL,
        temperature=TEMPERATURE,
        response_model=response_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )
    return resp

## Rule atomicity

Split into atomic rules.

In [ ]:
class ParagraphData(BaseModel):
    id: str = Field(..., description="The id of the given paragraph.")
    classification: str = Field(..., description="Classification of paragraph according to the definition of atomic rule.")
    text: str = Field(..., description="The text of the given paragraph.")
    atomic_paragraphs: List[str] = Field(..., description="List of atomic paragraphs.")

class ParagraphDataset(BaseModel):
    paragraphs: List[ParagraphData] = []

In [ ]:
system_prompt = """
Definition of atomic rule: Atomic rule is a rule that must be either fully complied with or completely contravened; partial compliance is not possible.

For each paragraph, classify the paragraph as atomic or non-atomic

Split the non-atomic paragraph into two or more atomic paragraphs. The words "or" and "and' is often an indicator that you should have 2 paragraphs instead of 1.

Here's an example of output:

[
  {
      "id": "P1",
      "classification": "atomic",
      "text": "some text",
      "atomic_paragraphs": ["some text"]
  },
  {
      "id": "P2",
      "classification": "non-atomic",
      "text": "some text and any other text",
      "atomic_paragraphs": ["some text", "some other text"]
  },
  {
      "id": "P3",
      "classification": "non-atomic",
      "text": "some text or any other text",
      "atomic_paragraphs": ["some text", "some other text"]
  },
  {
      "id": ...,
      "classification": ...,
      "text": ...,
      "atomic_paragraphs": ...
  }
]

# raw rules:
"""

In [ ]:
document1 = """
P1. “The reference to title for the land being transferred must be stated.”
P2. “The full name of the transferor must be stated and must be identical to the name of the registered proprietor as shown on the Register.”
P3. “The consideration is optional. A consideration stated must agree with the purchase price stated in the Notice of Sale form.”
P4. "A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment."
P5. If the Secretary certifies that the Commission was served with process, pleadings, or other papers pursuant to [paragraph (a)(1)](https://www.ecfr.gov/current/title-17/section-275.0-2#p-275.0-2(a)(1)) of this section and forwarded these documents to a named party pursuant to [paragraph (a)(2)](https://www.ecfr.gov/current/title-17/section-275.0-2#p-275.0-2(a)(2)) of this section, this certification constitutes evidence of service upon that party.
P6. All filings required to be made electronically with the Investment Adviser Registration Depository (“IARD”) shall, unless otherwise provided by the rules and regulations in this part, be deemed to have been filed with the Commission upon acceptance by the IARD. Filings required to be made through the IARD on a day that the IARD is closed shall be considered timely filed with the Commission if filed with the IARD no later than the following business day.
P7. You must count an owner as a client if you provide investment advisory services to the owner separate and apart from the investment advisory services you provide to the legal organization, provided, however, that the determination that an owner is a client will not affect the applicability of this section with regard to any other owner.
P8. To apply for registration with the Commission as an investment adviser, you must complete Form ADV (17 CFR 279.1) by following the instructions in the form and you must file Part 1A of Form ADV, the firm brochure(s) required by Part 2A of Form ADV and Form CRS required by Part 3 of Form ADV electronically with the Investment Adviser Registration Depository (IARD) unless you have received a hardship exemption under § 275.203-3. You are not required to file with the Commission the brochure supplements required by Part 2B of Form ADV.
P9. You must pay FINRA (the operator of the IARD) a filing fee. The Commission has approved the amount of the filing fee. No portion of the filing fee is refundable. Your completed application for registration will not be accepted by FINRA, and thus will not be considered filed with the Commission, until you have paid the filing fee.
P10. You must file Form ADV-W (17 CFR 279.2) to withdraw from investment adviser registration with the Commission (or to withdraw a pending registration application).
P11. If you are registered with a state securities authority, you must apply for registration with the Commission within 90 days of filing an annual updating amendment to your Form ADV reporting that you are eligible for SEC registration and are not relying on an exemption from registration under sections 203(l) or 203(m) of the Act (15 U.S.C. 80b-3(l), (m)).
P12. If you are registered under the Act as an investment adviser, you must deliver Form CRS, required by Part 3 of Form ADV [17 CFR 279.1], to each retail investor.
"""

True values

In [ ]:
true_values = [
  {
    "id": "P1",
    "classification": "atomic",
    "text": "The reference to title for the land being transferred must be stated.",
    "atomic_paragraphs": [
      "The reference to title for the land being transferred must be stated."
    ]
  },
  {
    "id": "P2",
    "classification": "non-atomic",
    "text": "The full name of the transferor must be stated and must be identical to the name of the registered proprietor as shown on the Register.",
    "atomic_paragraphs": [
      "The full name of the transferor must be stated.",
      "The full name of the transferor must be identical to the name of the registered proprietor as shown on the Register."
    ]
  },
  {
    "id": "P3",
    "classification": "non-atomic",
    "text": "The consideration is optional. A consideration stated must agree with the purchase price stated in the Notice of Sale form.",
    "atomic_paragraphs": [
      "The consideration is optional.",
      "A consideration stated must agree with the purchase price stated in the Notice of Sale form."
    ]
  },
  {
    "id": "P4",
    "classification": "non-atomic",
    "text": "A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment.",
    "atomic_paragraphs": [
      "A payment is a cash payment.",
      "A payment is a credit card payment.",
      "A payment is an electronic funds transfer payment."
    ]
  },
  {
    "id": "P5",
    "classification": "non-atomic",
    "text": "If the Secretary certifies that the Commission was served with process, pleadings, or other papers pursuant to [paragraph (a)(1)](https://www.ecfr.gov/current/title-17/section-275.0-2#p-275.0-2(a)(1)) of this section and forwarded these documents to a named party pursuant to [paragraph (a)(2)](https://www.ecfr.gov/current/title-17/section-275.0-2#p-275.0-2(a)(2)) of this section, this certification constitutes evidence of service upon that party.",
    "atomic_paragraphs": [
      "If the Secretary certifies that the Commission was served with process pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party.",
      "If the Secretary certifies that the Commission was served with pleadings pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party.",
      "If the Secretary certifies that the Commission was served with other papers pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party."
    ]
  },
  {
    "id": "P6",
    "classification": "non-atomic",
    "text": "All filings required to be made electronically with the Investment Adviser Registration Depository (“IARD”) shall, unless otherwise provided by the rules and regulations in this part, be deemed to have been filed with the Commission upon acceptance by the IARD. Filings required to be made through the IARD on a day that the IARD is closed shall be considered timely filed with the Commission if filed with the IARD no later than the following business day.",
    "atomic_paragraphs": [
      "All filings required to be made electronically with the Investment Adviser Registration Depository (“IARD”) shall, unless otherwise provided by the rules and regulations in this part, be deemed to have been filed with the Commission upon acceptance by the IARD.",
      "Filings required to be made through the IARD on a day that the IARD is closed shall be considered timely filed with the Commission if filed with the IARD no later than the following business day."
    ]
  },
  {
    "id": "P7",
    "classification": "non-atomic",
    "text": "You must count an owner as a client if you provide investment advisory services to the owner separate and apart from the investment advisory services you provide to the legal organization, provided, however, that the determination that an owner is a client will not affect the applicability of this section with regard to any other owner.",
    "atomic_paragraphs": [
      "You must count an owner as a client if you provide investment advisory services to the owner separate and apart from the investment advisory services you provide to the legal organization.",
      "The determination that an owner is a client will not affect the applicability of this section with regard to any other owner."
    ]
  },
  {
    "id": "P8",
    "classification": "non-atomic",
    "text": "To apply for registration with the Commission as an investment adviser, you must complete Form ADV (17 CFR 279.1) by following the instructions in the form and you must file Part 1A of Form ADV, the firm brochure(s) required by Part 2A of Form ADV and Form CRS required by Part 3 of Form ADV electronically with the Investment Adviser Registration Depository (IARD) unless you have received a hardship exemption under § 275.203-3. You are not required to file with the Commission the brochure supplements required by Part 2B of Form ADV.",
    "atomic_paragraphs": [
      "To apply for registration with the Commission as an investment adviser, you must complete Form ADV (17 CFR 279.1) by following the instructions in the form.",
      "You must file Part 1A of Form ADV electronically with the Investment Adviser Registration Depository (IARD) unless you have received a hardship exemption under § 275.203-3.",
      "You must file the firm brochure(s) required by Part 2A of Form ADV electronically with the IARD unless you have received a hardship exemption under § 275.203-3.",
      "You must file Form CRS required by Part 3 of Form ADV electronically with the IARD unless you have received a hardship exemption under § 275.203-3.",
      "You are not required to file with the Commission the brochure supplements required by Part 2B of Form ADV."
    ]
  },
  {
    "id": "P9",
    "classification": "non-atomic",
    "text": "You must pay FINRA (the operator of the IARD) a filing fee. The Commission has approved the amount of the filing fee. No portion of the filing fee is refundable. Your completed application for registration will not be accepted by FINRA, and thus will not be considered filed with the Commission, until you have paid the filing fee.",
    "atomic_paragraphs": [
      "You must pay FINRA (the operator of the IARD) a filing fee.",
      "The Commission has approved the amount of the filing fee.",
      "No portion of the filing fee is refundable.",
      "Your completed application for registration will not be accepted by FINRA until you have paid the filing fee.",
      "Your application will not be considered filed with the Commission until you have paid the filing fee."
    ]
  },
  {
    "id": "P10",
    "classification": "non-atomic",
    "text": "You must file Form ADV-W (17 CFR 279.2) to withdraw from investment adviser registration with the Commission (or to withdraw a pending registration application).",
    "atomic_paragraphs": [
      "You must file Form ADV-W (17 CFR 279.2) to withdraw from investment adviser registration with the Commission.",
      "You must file Form ADV-W (17 CFR 279.2) to withdraw a pending registration application."
    ]
  },
  {
    "id": "P11",
    "classification": "non-atomic",
    "text": "If you are registered with a state securities authority, you must apply for registration with the Commission within 90 days of filing an annual updating amendment to your Form ADV reporting that you are eligible for SEC registration and are not relying on an exemption from registration under sections 203(l) or 203(m) of the Act (15 U.S.C. 80b-3(l), (m)).",
    "atomic_paragraphs": [
      "If you are registered with a state securities authority, you must apply for registration with the Commission within 90 days of filing an annual updating amendment to your Form ADV reporting that you are eligible for SEC registration.",
      "If you are registered with a state securities authority, you must apply for registration with the Commission within 90 days of filing an annual updating amendment to your Form ADV reporting that you are not relying on an exemption from registration under sections 203(l) or 203(m) of the Act."
    ]
  },
  {
    "id": "P12",
    "classification": "atomic",
    "text": "If you are registered under the Act as an investment adviser, you must deliver Form CRS, required by Part 3 of Form ADV [17 CFR 279.1], to each retail investor.",
    "atomic_paragraphs": [
      "If you are registered under the Act as an investment adviser, you must deliver Form CRS, required by Part 3 of Form ADV [17 CFR 279.1], to each retail investor."
    ]
  }
]

labels = ["atomic", "non-atomic"]

Query LLM

In [ ]:
response=query_llm(user_prompt=document1, system_prompt=system_prompt, response_model=ParagraphDataset)

Response

In [ ]:
response.paragraphs

### Validation

Set predictions.

In [ ]:
# Predictions and True values
predictions = response.model_dump()

In [ ]:
# Convert to DataFrames
pred_df = pd.DataFrame(predictions["paragraphs"])
true_df = pd.DataFrame(true_values)

# Merge the two datasets on paragraph_id
merged_df = pd.merge(pred_df, true_df, on="id")

In [ ]:
merged_df

Compute confusion matrix and precision, recall, and accuracy.

In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(merged_df['classification_y'], merged_df['classification_x'], labels=labels)

# Compute precision, recall, and accuracy
precision = precision_score(merged_df['classification_y'], merged_df['classification_x'], average='weighted')
recall = recall_score(merged_df['classification_y'], merged_df['classification_x'], average='weighted')
accuracy = accuracy_score(merged_df['classification_y'], merged_df['classification_x'])
f1 = f1_score(merged_df['classification_y'], merged_df['classification_x'], average='weighted')

disp = ConfusionMatrixDisplay.from_predictions(merged_df['classification_y'], merged_df['classification_x'], labels=labels)

print(f"precison: {precision}, recall: {recall}, accuracy: {accuracy}, f1_score: {f1}")

## Classify (P6)

### Classify rule type

In [ ]:
class ParagraphData(BaseModel):
    type: str = Field(..., description="Type of rule according to the definition.")
    text: str = Field(..., description="The text of the given paragraph.")
    id: str = Field(..., description="The id of the given paragraph.")

class ParagraphDataset(BaseModel):
    paragraphs: List[ParagraphData] = []

prompt a

In [ ]:
prompt_a = """
You are an expert in SBVR (Semantics of Business Vocabulary and Business Rules). You'll be asked to classify paragraphs into types of rules. You'll be given a definition of the types of rules.

The best way to establish this is by a process of elimination:
  1. Definitional: If the rule cannot be violated;
  2. Party: If the outcome of testing the rule depends on who is involved (i.e., if one person performs the action, the rule is complied with, but if another person performs the action, the rule is contravened);
  3. Data: If the rule is concerned only with the presence or absence of one or more data items, how many instances of a data item, or the content of one or more data items;
  4. Activity: If none of the above.

Definition
  1. Definitional: Definitional rules constrain how we define constructs created or used by the organization or the industry within which it operates;
  2. Party: Party rules (all of which are operative rules) restrict the parties who can perform processes or activities or play roles.
  3. Data: Data rules (all of which are operative rules) constrain the data included in a transaction (a form or message) or a persistent data set;
  4. Activity: Activity rules (all of which are operative rules) constrain the operation of one or more business processes or other activities;

 If the query is invalid, return an empty classification.

 Here's an example
 [
    {
        "id": "some id",
        "text": "some text",
        "type": "Definitional"
    },
    {
        "id": ...
        "text": ...
        "type": ...
    },

]

Here's the paragraphs you'll need to classify:
"""

prompt b

In [ ]:
prompt_b = """
You are an expert in SBVR (Semantics of Business Vocabulary and Business Rules). You'll be asked to classify paragraphs into types of rules. You'll be given a definition of types of rules.

Definition
  1. Definitional: Definitional rules constrain how we define constructs created or used by the organization or the industry within which it operates;
  2. Party: Party rules restrict the parties who can perform processes or activities or play roles.
  3. Data: Data rules constrain the data included in a transaction (a form or message) or a persistent data set;
  4. Activity: Activity rules constrain the operation of one or more business processes or other activities;

 Here's an example of the output in JSON:
 [
    {
        "id": "some id",
        "text": "some text",
        "type": "Definitional"
    },
    {
        "id": ...
        "text": ...
        "type": ...
    },

]

Here's the paragraphs you'll need to classify:
"""

Paragraphs to classify:

In [ ]:
document1 = """
'R70', 'A senior passenger is by definition a passenger whose age is at least 70 years at the time of travel.',
'R73', 'End of financial year is by definition June 30.',
'R76', 'A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment.',
'R85', '1 ft is by definition equal to 12 in.',
'R91', 'Each flight booking confirmation must specify exactly one travel class for each flight.',
'R98', 'Each flight booking confirmation must specify exactly one of the following: a postal address, an e-mail address, or a fax number.',
'R99', 'A flight booking request for a one-way journey must not specify a return date.',
'R130', 'Online check-in for a flight may occur only during the 24 h before the departure time of that flight.',
'R135', 'A driver must not operate any vehicle if that driver is intoxicated.',
'R140', 'A person may travel alone only if the age of that person is at least 2 years.',
'R78', 'The status of an employee is by definition one of the following: probational, permanent, or temporary.',
'R349', 'Each flight booking confirmation for an international journey must specify for each passenger specified in that flight booking request a passport number or a visa number but not both.',
'R75', 'A person is by definition either an adult or a minor.',
'R93', 'Each combination of departure date, flight number, and departure city must be allocated exactly one departure time.',
'R96', 'Each flight booking confirmation must specify a mobile phone number, an e-mail address, or both.',
'R143', 'A person may be rostered on a flight crew only if that person holds an airline transport pilot license that is current and a type endorsement that is current for each aircraft type to be flown by that flight crew.',
'R142', 'A passenger may be allocated to a seat in an exit row only if that passenger is able to open an aircraft door.',
'R77', 'The gender of a person is by definition either male or female.',
'R348', 'Each customer complaint must specify a mobile phone number, an e-mail address, or both.'
"""

True values

In [ ]:
true_values = [
    {"id": "R70", "type": "Definitional"},
    {"id": "R73", "type": "Definitional"},
    {"id": "R76", "type": "Definitional"},
    {"id": "R85", "type": "Definitional"},
    {"id": "R91", "type": "Data"},
    {"id": "R98", "type": "Data"},
    {"id": "R99", "type": "Data"},
    {"id": "R130", "type": "Activity"},
    {"id": "R135", "type": "Activity"},
    {"id": "R140", "type": "Party"},
    {"id": "R78", "type": "Definitional"},
    {"id": "R349", "type": "Data"},
    {"id": "R75", "type": "Definitional"},
    {"id": "R93", "type": "Data"},
    {"id": "R96", "type": "Data"},
    {"id": "R143", "type": "Party"},
    {"id": "R142", "type": "Party"},
    {"id": "R77", "type": "Definitional"},
    {"id": "R348", "type": "Data"}
]
labels = ["Definitional", "Data", "Activity", "Party"]

Choose prompt to test:

In [ ]:
system_prompt = prompt_b

system_prompt

In [ ]:
response = query_llm(user_prompt=document1, system_prompt=system_prompt, response_model=ParagraphDataset)

In [ ]:
response.paragraphs

#### Validation


Set predictions.

In [ ]:
# Predictions and True values
predictions = response.model_dump()

Convert to dataframe.

In [ ]:
# Convert to DataFrames
pred_df = pd.DataFrame(predictions["paragraphs"])
true_df = pd.DataFrame(true_values)

# Merge the two datasets on paragraph_id
merged_df = pd.merge(pred_df, true_df, on="id")

In [ ]:
merged_df

Compute confusion matrix and precision, recall, and accuracy.

In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(merged_df['type_y'], merged_df['type_x'], labels=labels)

# Compute precision, recall, and accuracy
precision = precision_score(merged_df['type_y'], merged_df['type_x'], average='weighted')
recall = recall_score(merged_df['type_y'], merged_df['type_x'], average='weighted')
accuracy = accuracy_score(merged_df['type_y'], merged_df['type_x'])
f1 = f1_score(merged_df['type_y'], merged_df['type_x'], average='weighted')

disp = ConfusionMatrixDisplay.from_predictions(merged_df['type_y'], merged_df['type_x'], labels=labels)

print(f"precison: {precision}, recall: {recall}, accuracy: {accuracy}, f1_score: {f1}")

### Classify rule sub-type

In [ ]:
class ParagraphData(BaseModel):
    type: str = Field(..., description="The type of the rule.")
    text: str = Field(..., description="The text of the given paragraph.")
    id: str = Field(..., description="The id of the given paragraph.")
    subtype: str = Field(..., description="The sub-type of the rule.")
    templates: list[str] = Field(..., description="The templates of the rule.")
    examples: list[str] = Field(..., description="The examples of the rule.")

class ParagraphDataset(BaseModel):
    paragraphs: List[ParagraphData] = []

In [ ]:
prompt_a = """
You are an expert in SBVR (Semantics of Business Vocabulary and Business Rules). You'll be asked to classify paragraphs into types of rules. You'll be given a definition of the types of rules.

Given the definition of types of rules, and their templates, classify each rule by their sub-types. see template syntax to interpret the templates.
# Template and subtemplate sintaxe
1. Each item enclosed in angle brackets (\< and \>) is a placeholder, in place of which any suitable text may be substituted. For example, any of the following may be substituted in place of \<operative rule statement subject\>:
	- a term: for example, "flight booking request",
	- a term followed by a qualifying clause: for example, "flight booking request for a one-way journey",
	- a reference to a combination of items: for example, "combination of enrollment date and graduation date", with or without a qualifying clause,
	- a reference to a set of items: for example, "set of passengers", with or without a qualifying clause.
2. Each pair of braces ({ and }) encloses a set of options (separated from each other by the bar symbol: |), one of which is included in the rule statement. For example,
	- each rule statement starts with either "Each" or "The";
	- a conditional clause in a rule statement is preceded by either "if" or "unless".
3. If a pair of braces includes a bar symbol immediately before the closing brace, the null option is allowed: that is, you can, if necessary, include none of the options at that point in the rule statement. For example, each rule statement may include or omit a conditional clause preceded by "if" or "unless".
4. Sets of options may be nested. For example, in each of the templates above
	- a conditional clause may be included or omitted,
	- if included, the conditional clause should be preceded by either "if" or "unless".
5. A further notation, introduced later in this section, uses square brackets to indicate that a syntactic element may be repeated indefinitely.
6. Any text not enclosed in either angle brackets or braces (i.e., "must", "not", "may", and "only") is included in every rule statement conforming to the relevant template.
7. Subtemplate has a designator before the symbol "::=". The text after the symbol "::=" can be substituted in place of any placeholder (in a template or subtemplate) that has the same designator.

# Definition
## 9.2 Definitional rules
Definitional rules constrains how we define a construct created or used by the organization or the industry within which it operates. Definitional rules can in turn be categorized as:
### 9.2.1 Formal term definitions:
A formal term definition defines a particular business term in a formal manner. They are categorized as:
#### 9.2.1.1 Formal intensional definitions
A formal intensional definition defines the subject business term using an intensional definition: one that cites both a hypernym (a term that refers to a superset of the set referred to by the original term) and the characteristics that distinguish members of the set referred to by the original term.
T7.
```template
{A|An} <term 1>
	{of {a|an} <term 2>| }
is by definition
{a|an|the} <term 3>
	<qualifying clause>.
```
#### 9.2.1.2 Formal extensional definitions
Formal extensional definition defines the subject business term by using an extensional definition: one that lists a complete set of hyponyms (terms that refer to subsets of the set referred to by the original term).
T8.
```template
{A|An} <term 1>
	{of {a|an} <term 2>| }
is by definition
[<article> <term 3>, or]
	{of that <term 2>| }.
```
#### 9.2.1.3 Symbolic literal definitions
A symbolic literal definition defines the subject business term using one or more literals.
T9.
```template
{<literal 1>|{A|An} <term 1>
	{of {a|an} <term 2>| }}
is by definition
{<literal 2>|
[<literal 3>, or] from a <literal 4> to the following <literal 5>}.
```
### 9.2.2 Categorization scheme enumerations
A categorization scheme enumeration defines the members of a categorization scheme that is both mutually exclusive and jointly exhaustive.
T10.
```template
{{A|An} <category attribute term>|
The <category attribute term>
	of {a |an} <entity class term>}
is by definition
{either <literal 1> or <literal 2>|
one of the following: [<literal 3>, or]}.
```
### 9.2.3 Category transition constraints
A category transition constraint specifies allowed or disallowed transitions between categories or statuses.
T11.
```template
A transition
	of the <category attribute term> of {a|an} <entity class term>
	from {<literal 1>| [<literal 2>, or]}
	to {<literal 3>| [<literal 4>, or]}
is by definition
impossible.
```
### 9.2.4 Complex concept structure rules
A complex concept structure rule defines a particular constraint on one or more components of a complex concept. They are categorized as:
#### 9.2.4.1 Complex concept cardinality rules
A complex concept cardinality rule defines the number of (or minimum and/or maximum number of) components of a particular type within a particular concept.
T12.
```template
{A|An} <term 1>
<verb phrase> by definition
{<cardinality>|at most <positive integer>} <term 2>
	{{for |in} {each|the} <term 3>| }.
```
#### 9.2.4.2 Complex concept equivalence rules
A complex concept equivalence rule defines a pair of components within a particular concept that are of necessity the same.
T13.
```template
The <term 1>
	<qualifying clause 1>
is by definition
the same as the <term 2>
	<qualifying clause 2>.
```
#### 9.2.4.3 Complex concept set constraints
A complex concept set constraint defines two sets of components within a particular concept that must be identical.
T14.
```template
The set of <term 1>
	<qualifying clause 1>
is by definition
the same as the set of <term 1>
	<qualifying clause 2>.
```
### 9.2.5 Valid value definitions
A valid value definition defines the valid values of a particular measure as a range or (occasionally) as a list of discrete values.
T15.
```template
{The| } <attribute term>
	{of {a|an} <entity class term>| }
is by definition
{<inequality operator> <literal 1>
	{and <inequality operator> <literal 2>| } |
	[<literal 3>, or]}.
```
### 9.2.6 Data calculation rules
A data calculation rule defines the algorithm or formula for a particular quantity or a conversion factor between two units. They are categorized as:
#### 9.2.6.1 Data calculation algorithms
A data calculation algorithm defines how a particular quantity or amount (whether for operational purposes, such as a fee, or for business intelligence purposes, such as a performance measure) is calculated.
T16.
```template
{The| } <attribute term>
	{of | for} {a|an} <entity class term>
	{<qualifying clause>| }
is by definition calculated as
<expression>.
```
#### 9.2.6.2 Conversion factor definitions
A conversion factor definition defines a conversion factor between two units of measurement.
T17.
```template
<literal 1>
is by definition {approximately | } equal to
<literal 2>.
```
### 9.2.7 Standard format definitions
A standard format definition defines the standard format for data items of a particular type in terms of individual characters and/or component data items.
T18.
```template
A valid <term>
is by definition composed of
<format definition>.
```
## 9.3 Data rules
Data rules (all of which are operative rules) constrains the data included in a transaction (a form or message) or a persistent dataset (e.g., a database record). Data rules can in turn be categorized as:
### 9.3.1 Data cardinality rules
A data cardinality rule requires the presence or absence of a data item and/or places a restriction on the maximum or minimum number of occurrences of a data item
#### 9.3.1.1 Mandatory data rules
A mandatory data rule mandates the presence of data:
##### 9.3.1.1.1 Mandatory data item rules
A mandatory data item rule requires that a particular data item be present.
T19.
```template
Each <transaction signifier>
must {specify|contain} <cardinality> <data item term>
	{{in| for} {each|the} <subform term> {(if any)| }
	{<qualifying clause>| } | }
{{if |unless} <conditional clause>| }.
```
##### 9.3.1.1.2 Mandatory option selection rules
A mandatory option selection rule requires that one of a set of pre-defined options be specified.
with two or more options:
T20.
```template
Each <transaction signifier>
must
{({if |unless} <conditional clause>) | }
specify whether {it |{the |each} <term>
	{<qualifying clause>| }}
<verb phrase> [<object>, or].
```
with a single option:
T21.
```template
Each <transaction signifier>
must
{({if |unless} <conditional clause>) | }
specify whether {or not| } {it |{the |each} <term>
	{<qualifying clause>| }}
<verb phrase> {<object>| }.
```
##### 9.3.1.1.3 Mandatory group rules:
A mandatory group rule requires that at least one of a group of data items be present.
two data items in the group:
T22.
```template
Each <transaction signifier>
must {specify|contain}
	{{in| for} {each|the} <subform term> {(if any)| }
	{<qualifying clause>| } | }
	{a|an} <data item term 1>, {a|an} <data item term 2>
	{, or|but not} both
{{if |unless} <conditional clause>| }.
```
more than two data items in the group:
T23.
```template
Each <transaction signifier>
must
{({if |unless} <conditional clause>) | }
{specify |contain}
	{{in| for} {each|the} <subform term> {(if any)| }
	{<qualifying clause>| } | }
	<cardinality> of the following:
	[<data item term>, or].
```
#### 9.3.1.2 Prohibited data rules
A prohibited data rule mandates the absence of some data item in a particular situation.
T24.
```template
{A|An} <transaction signifier>
must not {specify |contain} a <data item term>
	{{in | for} {any|the} <subform term> {(if any)| }
	{<qualifying clause>| } | }
{{if |unless} <conditional clause>| }.
```
#### 9.3.1.3 Maximum cardinality rules
A maximum cardinality rule places an upper limit (usually but not necessarily one) on how many instances of a particular data item there may be.
T25.
```template
{A|An} <transaction signifier>
must not {specify |contain} more than <positive integer>
	<data item term>
	{{in | for} {any one|the} <subform term> {(if any)| }
	{<qualifying clause>| } | }
{{if |unless} <conditional clause>| }.
```
#### 9.3.1.4 Multiple data rules
A multiple data rule mandates the presence of two or more instances of a particular data item in a particular situation.
T19.
```template
Each <transaction signifier>
	must {specify|contain} <cardinality> <data item term>
	{{in| for} {each|the} <subform term> {(if any)| }
	{<qualifying clause>| } | }
{{if |unless} <conditional clause>| }.
```

these rule statements `<cardinality>` may only take one of the following forms:
1. exactly `<positive integer>`, where `<positive integer>` is at least two;
2. at least `<positive integer>`, where `<positive integer>` is at least two;
3. at least `<positive integer 1>` and at most `<positive integer 2>`, where `<positive integer 1>` is at least two.
#### 9.3.1.5 Dependent cardinality rules
A dependent cardinality rule mandates how many of a particular data item must be present based on the value of another data item.
T26.
```template
The number of <data item term 1>
	{specified|contained}
	{{in| for} {the|each} <subform term> {(if any) | } | }
	in each <transaction signifier>
must be {{no|} {more|less} than|equal to} the <data item term 2>
	{<qualifying clause>| }
{{if |unless} <conditional clause>| }.
```
### 9.3.2 Data content rules
A data content rule places a restriction on the values contained in a data item or set of data items (rather than whether they must be present and how many there may or must be).
#### 9.3.2.1 Value set rules
A value set rule requires either: that the content of a data item be (or not be) one of a particular set of values (either a fixed set, or a set that may change over time), or; that the content of a combination of data items match or not match a corresponding combination in a set of records;
##### 9.3.2.1.1 Value set rules constraining single data items
T27.
```template
{The|Each} <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be
	{{other than| } one of the <term> <qualifying clause>| [<literal>, or]}
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.1.2 Value set rules constraining combinations of data items
T28.
```template
{The|Each} combination of [<data item term 1>, and] {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be one of the combinations of [<data item term 2>, and]
	{<qualifying clause>| }
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.2. Range rules
A range rule requires that the content of a data item be a value within a particular inclusive or exclusive single-bounded or double-bounded range.
T29.
```template
{The|Each} <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be <inequality operator> <object> {and <inequality operator> <object>| }
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.3 Equality rules
An equality rule requires that the content of a data item be the same as or not the same as that of some other data item.
T30.
```template
{The|Each} <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be <equality operator> <object>
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.4 Uniqueness constraints
A uniqueness constraint requires that the content of a data item (or combination or set of data items) be different from that of the corresponding data item(s) in the same or other records or transactions;
##### 9.3.2.4.1 Uniqueness constraints constraining single data items.
T31.
```template
{The|Each} <data item term 1> {(if any)| }
	<verb part> {the <subform term 1> {(if any)| }
	in|} each <transaction signifier 1>
	{<qualifying clause 1>| }
must be different from the <data item term 1>
	<verb part> {{the |any other} <subform term 1> {(if any)| }
	in| } {that |any other} <transaction signifier 1>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.4.2 Uniqueness constraints constraining combinations of data items
T32.
```template
{The|Each} combination of [<data item term 1>, and] {(if any)| }
	<verb part> {the <subform term 1> {(if any)| }
	in| } each <transaction signifier 1>
	{<qualifying clause 1>| }
must be different from the combination of [<data item term 1>, and]
	<verb part> {{the|any other} <subform term 1> {(if any) | }
	in| } {that |any other} <transaction signifier 1>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.4.3 Uniqueness constraints constraining sets of data items
T33.
```template
{The|Each} set of <data item term 1> {(if any)| }
	<verb part> {the <subform term 1> {(if any)| }
	in|} each <transaction signifier 1>
	{<qualifying clause 1>| }
must be different from the set of <data item term 1>
	<verb part> {{the |any other} <subform term 1> {(if any)| }
	in| } {that |any other} <transaction signifier 1>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.5 Data consistency rules
A data consistency rule requires the content of multiple data items to be consistent with each other, other than as provided for by a value set rule, range rule, or equality rule;
##### 9.3.2.5.1 Data consistency rules constraining a combination of data items
T34.
```template
{The|Each} combination of [<data item term>, and] {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
	{<qualifying clause>| }
must be such that <conditional clause 1>
{{if |unless} <conditional clause 2>| }.
```
##### 9.3.2.5.2 Data consistency rules constraining a set function
T35.
```template
The <set function> of {the| } <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
	{<qualifying clause>| }
must be {<inequality operator>|<equality operator>} <object>
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.5.3 Data consistency rules constraining a set
T36.
```template
{The|Each} set of <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
	{<qualifying clause 1>| }
must {be {the same as| different from} |include} the set of <term>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.6 Temporal data constraints
A temporal data constraint constrains one or more temporal data items (data items that represent time points or time periods). There are various subcategories of temporal constraint:
##### 9.3.2.6.1 Simple temporal data constraints
A simple temporal data constraint requires that a particular date or time fall within a certain temporal range.
T29.
```template
{The|Each} <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be <temporal inequality operator> <object> {and <temporal inequality operator> <object>| }
{{if |unless} <conditional clause>| }.
```
R373.
```example
The departure time of the outgoing flight
	specified in each flight booking confirmation
	that is made online
must be no earlier than 3 h
	after the booking confirmation time
	of that flight booking confirmation.
```
##### 9.3.2.6.2 Temporal data non-overlap constraints
Temporal data non-overlap constraint requires that the time periods specified in a set of records do not overlap each other.
T37.
```template
{The|Each} <time period term 1> {(if any)| }
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in each <transaction signifier 1>
	{<qualifying clause 1>| }
must not overlap the <time period term 1>
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in any other <transaction signifier 1>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.6.3 Temporal data completeness constraints
A temporal data completeness constraint requires that the time periods specified in a set of records be contiguous and between them completely span some other time period.
T38.
```template
Each <time period term 1>
	within the <time period term 2> {(if any)| }
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in each <transaction signifier 1>
	{<qualifying clause 1>| }
must be within the <time period term 3>
	specified {{in| for} {the|each} <subform term 2> {(if any)| } | }
	in <cardinality> <transaction signifier 2>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.6.4 Temporal data inclusion constraints
A temporal data inclusion constraint requires that the time periods specified in a set of records do not fall outside some other time period.
T38.
```template
Each <time period term 1>
	within the <time period term 2> {(if any)| }
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in each <transaction signifier 1>
	{<qualifying clause 1>| }
must be within the <time period term 3>
	specified {{in| for} {the|each} <subform term 2> {(if any)| } | }
	in <cardinality> <transaction signifier 2>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.6.5 Temporal single record constraints
A temporal single record constraint requires that a temporal state of affairs be recorded using a single record rather than multiple records.
single data item is involved:
T39.
```template
{The|Each} <data item term 1> {(if any)| }
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in each <transaction signifier 1>
must be different from the <data item term 1>
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in the latest of the earlier <transaction signifier 1>
{{if |unless} <conditional clause>| }.
```
combination of data items is involved:
T40.
```template
{The|Each} combination of [<data item term 1>, and] {(if any)| }
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in each <transaction signifier 1>
must be different from the combination of [<data item term 1>, and]
	specified {{in| for} {the|each} <subform term 1> {(if any)| } | }
	in the latest of the earlier <transaction signifier 1>
{{if |unless} <conditional clause>| }.
```
##### 9.3.2.6.6 Day type constraints
A day type constraint restricts a date to one or more days of the week or a particular type of day such as a working day (typically but not necessarily any day other than a Saturday, Sunday, or public holiday).
T41.
```template
{The|Each} <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be a {<term>|<literal 1>| [<literal 2>, or]}
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.7 Spatial data constraints
A spatial data constraint prescribes or prohibits relationships between data items representing spatial properties (points, line segments or polygons).
T41.
```template
{The|Each|A|An} <spatial term 1> {(if any)| }
	<qualifying clause 1>
must {not| } <spatial operator> the <spatial term 2>
	<qualifying clause 2>
{{if |unless} <conditional clause>| }.
```
#### 9.3.2.8 Data item format rules
A data item format rule specifies the required format of a data item.
T43.
```template
The <data item term> {(if any)| }
	specified {{in| for} {the|each} <subform term> {(if any)| } | }
	in each <transaction signifier>
must be {represented using| } a valid <term>
{{if |unless} <conditional clause>| }.
```
### 9.3.3 Data update rules
A data update rule either prohibits update of a data item or places restrictions on the new value of a data item in terms of its existing value. There are three subcategories of data update rule:
#### 9.3.3.1 Data update prohibition rules
A data update prohibition rule prohibits update of a particular data item or set of data items.
non-transferable relationships:
T44.
```template
{A|An} <transaction signifier 1>
must not be transferred
	from one <transaction signifier 2> to another <transaction signifier 2>
{{if |unless} <conditional clause>| }.
```
other data update:
T45:
```template
{The|A|An} <data item term> {(if any)| }
	{{in |for} {any|the} <subform term> {(if any)| } | }
	{in|of} a <transaction signifier>
must not be updated
{{if |unless} <conditional clause>| }.
```
#### 9.3.3.2 State transition constraints
A state transition constraint limits the changes in a data item to a set of valid transitions.
T46.
```template
The <data item term> {(if any)| }
	{{in |for} {any|the} <subform term> {(if any)| } | }
	{in|of} a <transaction signifier>
may be updated to {<literal 1>| [<literal 2>, or]}
only if <conditional clause>.
```
#### 9.3.3.3 Monotonic transition constraints
A monotonic transition constraint requires that a numeric value either only increase or only decrease.
T47.
```template
The <data item term> {(if any)| }
	{{in| for} {any|the} <subform term> {(if any)| } | }
	{in |of} a <transaction signifier>
must not be {increased|decreased}
{{if |unless} <conditional clause>| }.
```
## 9.4 Activity rules
Activity rules (all of which are operative rules) constrains the operation of one or more business processes or other activities. Activity rules can in turn be categorized as:
### 9.4.1 Activity restriction rules
An activity restriction rule restricts a business process or other activity in some way. There are various subcategories of activity restriction rules:
#### 9.4.1.1 Rules restricting when an activity can occur
Many activity restriction rules place time restrictions on activities.
##### 9.4.1.1.1 Activity time limit rules
An activity time limit rule restricts a business process or other activity to within a particular time period.
T48.
```template
{The| } <process term> {of | for} {a|an} <object term>
	{<qualifying clause>| }
{must {not| } occur|may occur only}
<time restriction 1> {{and| or} <time restriction 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.4.1.1.2 Activity exclusion period rules
An activity exclusion period rule prohibits a business process or other activity during a particular time period.
T49.
```template
{Each|A|An} <term>
	{<qualifying clause 1>| }
{must {not| } <verb phrase 1> {<object 1>| }
	{<qualifying clause 2>| } |
may <verb phrase 2> {<object 2>| }
	{<qualifying clause 3>| } only}
<time restriction 1> {{and| or} <time restriction 2>| }
{{if |unless} <conditional clause>| }.
```
##### 9.4.1.1.3 Activity obligation rule
An activity obligation rule requires a business process or other activity to occur either within a maximum time after a particular event (such as the completion of some other process) or as soon as practical after a particular event.
T49.
```template
{Each|A|An} <term>
	{<qualifying clause 1>| }
{must {not| } <verb phrase 1> {<object 1>| }
	{<qualifying clause 2>| } |
may <verb phrase 2> {<object 2>| }
	{<qualifying clause 3>| } only}
<time restriction 1> {{and| or} <time restriction 2>| }
{{if |unless} <conditional clause>| }.
```
#### 9.4.1.2 Activity pre-condition rules
An activity pre-condition rule prohibits a business process or other activity unless some other activity or event has previously occurred or some prerequisite condition exists.
T50.
```template
{A|An} <subject term>
	{<qualifying clause>| }
may <verb phrase> {<object>| }
only {<time restriction>| if <conditional clause>}.
```
#### 9.4.1.3 Activity prohibition rules
An activity prohibition rule prohibits a business process or other activity if some event or other process has previously occurred or some dangerous or illegal condition exists.
T51.
```template
{A|An} <subject term>
	{<qualifying clause>| }
must not <verb phrase> {<object>| }
if <conditional clause>.
```
#### 9.4.1.4 Information retention rules
An information retention rule defines the minimum period for which a particular type of information is retained.
T49.
```template
{Each|A|An} <term>
	{<qualifying clause 1>| }
{must {not| } <verb phrase 1> {<object 1>| }
	{<qualifying clause 2>| } |
may <verb phrase 2> {<object 2>| }
	{<qualifying clause 3>| } only}
<time restriction 1> {{and| or} <time restriction 2>| }
{{if |unless} <conditional clause>| }.
```
#### 9.4.1.5 Activity conflict rules
An activity conflict rule restricts the simultaneous occurrence of multiple processes or other activities.
```
R136.
```example
R136. A folder
must not be renamed
while any file within that folder is open for editing.
```
### 9.4.2 Process decision rules
A process decision rule determines what action a business process or device is to take in specific situations;
9.4.3 Activity obligation rules: An activity obligation rule requires a business process or other activity to occur either within a maximum time after a particular event (such as the completion of some other process) or when particular conditions apply.
T52.
```template
Each <actor term>
must <verb phrase> {<object>| }
	{<qualifying clause>| }
{{if |unless} <conditional clause>| }.
```
## 9.5 Party rules
Party rules (all of which are operative rules) restricts the parties who can perform a process or activity or play a role. Party rules can in turn be categorized as
### 9.5.1 Party restriction rules
A party restriction rule places restrictions on who can perform some processes or activities or play some roles, based on age, some other physical characteristic or capability, or training, testing, and certification in the appropriate skills.
T53.
```template
A <party signifier 1>
	{<qualifying clause>| }
may <predicate 1>
only if {the <attribute signifier> of| } that <party signifier 1>
	<predicate 2>.
```
### 9.5.2 Role separation and binding rules
A role separation rule prohibits the same party from performing two activities.
T54.
```template
The <party signifier 1>
	<qualifying clause 1>
must {not| } be {the same|one of the} <party signifier 1>
	<qualifying clause 2>
{{if |unless} <conditional clause>| }.
```
### 9.5.3 Information access rules
An information access rule defines who can view, create, or update particular information.
T55.
```template
{The|A|An} <information signifier>
	<qualifying clause>
may be <information access process> by
only {<object 1>| [<object 2>, or]}
{{if |unless} <conditional clause>| }.
```
### 9.5.4 Responsibility rules
A responsibility rule defines who is responsible for performing a particular process or liable for a particular fee, duty, or tax.
T56.
```template
{The|A|An| } <responsibility signifier>
	{<qualifying clause 1>| }
must <verb phrase> {the |a|an} <party signifier>
	{<qualifying clause 2>| }
{{if |unless} <conditional clause>| }.
```
### Subtemplates
S1.
```subtemplate
<operative rule statement subject>::=
{<term>|combination of [<term>, and]|set of <term>}
{<qualifying clause>|}
```
S2.
```subtemplate
<article>::= {a|an|the}
```
S3.
```subtemplate
<cardinality>::=
{exactly|at least {<positive integer 1> and at most| }}
<positive integer 2>
```
S4.
```subtemplate
<determiner>::=
{<article>|each|that |those|
<cardinality>|at most <positive integer>|
```
S5.
```subtemplate
<set function>::=
{number |sum| total |maximum|minimum|average|mean|median|
latest | earliest}
```
S6.
```subtemplate
<inequality operator>::=
{{no|} {more|less | later | earlier} than|
at {least |most} <literal> {more| later} than|
{no|} {later | earlier} than <literal> {after |before}}
```
S7.
```subtemplate
<equality operator>::=
{the same as| different from|equal to|unequal to}
```
S8.
```subtemplate
<transaction signifier>::=
{<term>|{record of a| } combination of [<term>, and]}
{<qualifying clause>| }
```
S9.
```subtemplate
<verb part>::=
{<participle>|<adjective>| } <preposition>
```
S10.
```
<predicate>::=
{{<verb phrase>| is {<equality operator>|<inequality operator>}}
<object>|
<verb phrase> {[<object>, and] | [<object>, or] | }}
```
S11.
```subtemplate
<object>::=
{{<determiner>|the <set function> of {<determiner>| } | } <term>
{<qualifying clause>| } |
{<determiner>| } <literal>}
```
S12.
```subtemplate
<expression>::=
{<object>|
<set function> of {<determiner>| } <term> {<qualifying clause>| } |
<expression> {plus|minus|multiplied by|divided by} <expression>|
{sum|product} of [<expression>, and] |
{square|cube} {root |} of <expression>}
```
S13.
```subtemplate
<conditional clause>::=
{{<determiner> <term> {<qualifying clause>| } |<expression>| it}
{<predicate>| [<predicate> and]| [<predicate> or]} |
[<conditional clause> and] | [<conditional clause> or]}
```
S14.
```subtemplate
<qualifying clause>::=
{{that |who} <verb phrase> {<object>| } |
<verb part> <object>|
other than {<object>| [<object>, or]} |
{<preposition> {which|whom}|whose} <conditional clause>|
{that |who} <verb phrase> {that | if |whether} <conditional clause>|
<and-qualifying clause>|
<or-qualifying clause>|
<both-and-qualifying clause>|
<either-or-qualifying clause>}
```
S15.
```subtemplate
<and-qualifying clause>::=
{that |who}
{[<verb phrase> {<object>| } and]|
is [<verb part> {<object>| } and]
<verb phrase> [<object> and]}
```
S16.
```subtemplate
<both-and-qualifying clause>::=
{that |who}
{both <verb phrase> {<object>| } and <verb phrase> {<object>| } |
{is |are} both <verb part> {<object>| } and <verb part> {<object>| } |
<verb phrase> both <object> and <object>}
```
S17.
```subtemplate
<or-qualifying clause>::=
{that |who}
{[<verb phrase> {<object>| } or] |
is [<verb part> {<object>| } or]
<verb phrase> [<object> or]}
```
S18.
```subtemplate
<either-or-qualifying clause>::=
{that |who}
{either <verb phrase> {<object>|} or <verb phrase> {<object>| } |
{is |are} either <verb part> {<object>|} or <verb part> {<object>| } |
<verb phrase> either <object> or <object>}
```
S19.
```subtemplate
<format definition>::=
{{exactly|at least |up to} <positive integer 1>|
from <positive integer 2> to <positive integer 3>} <term>
{followed by <format definition>| }
```
S20.
```subtemplate
<spatial operator>::=
{overlap|be within|enclose|span|intersect |meet|be on}
```
S21.
```subtemplate
<time restriction>::=
{at any time| }
{{before| after |during| until |within| {no|} {earlier | later} than} <object>| }
{{before| after |during| until |within|on} <object>|
{before| after |while| until} <conditional clause>}
```
S22.
```subtemplate
<information access process>::=
{viewed|created|updated|deleted}
```

Return a JSON object with the following fields: paragraph, rule_type.

 If the query is invalid, return an empty classification.

 Here's an example
 [
    {
        "id": "some id",
        "text": "some text",
        "type": "Definitional"
        "subtype": "Formal intensional definitions",
        "templates": ["T7"],
        "examples": ["R70"]
    },
    {
        "id": ...
        "text": ...
        "type": ...
        "subtype": ...
        "templates": ...
        "examples": ...
    },

]

Here's the paragraphs you'll need to classify:
"""

In [ ]:
prompt_b = """
You are an expert in SBVR (Semantics of Business Vocabulary and Business Rules). You'll be asked to classify paragraphs into types of rules. You'll be given a definition of the types of rules.

Given the definition of types of rules, classify each rule by following subtypes.

Hints:
- If a section has no templates, look in the next section. E.g. 9.2.1 has no template, so look at 9.2.1.1, it has a template T7;
- If a section has more tha one template, check when apply each template. E.g. 9.2.1.2 has two templates, so look at 9.2.1.2.1, it has a template T8;
- The subtype is the section name.

# Definition

## 9.2 Definitional rules
Definitional rules constrains how we define a construct created or used by the organization or the industry within which it operates. Definitional rules can in turn be categorized as:

### 9.2.1 Formal term definitions:
A formal term definition defines a particular business term in a formal manner. They are categorized as:

#### 9.2.1.1 Formal intensional definitions
A formal intensional definition defines the subject business term using an intensional definition: one that cites both a hypernym (a term that refers to a superset of the set referred to by the original term) and the characteristics that distinguish members of the set referred to by the original term.
Template: T7

#### 9.2.1.2 Formal extensional definitions
Formal extensional definition defines the subject business term by using an extensional definition: one that lists a complete set of hyponyms (terms that refer to subsets of the set referred to by the original term).
Template: T8

#### 9.2.1.3 Symbolic literal definitions
A symbolic literal definition defines the subject business term using one or more literals.
Template: T9

### 9.2.2 Categorization scheme enumerations
A categorization scheme enumeration defines the members of a categorization scheme that is both mutually exclusive and jointly exhaustive.
Template: T10

### 9.2.3 Category transition constraints
A category transition constraint specifies allowed or disallowed transitions between categories or statuses.
Template: T11

### 9.2.4 Complex concept structure rules
Template: A complex concept structure rule defines a particular constraint on one or more components of a complex concept. They are categorized as:

#### 9.2.4.1 Complex concept cardinality rules
A complex concept cardinality rule defines the number of (or minimum and/or maximum number of) components of a particular type within a particular concept.
Template: T12

#### 9.2.4.2 Complex concept equivalence rules
A complex concept equivalence rule defines a pair of components within a particular concept that are of necessity the same.
Template: T13

#### 9.2.4.3 Complex concept set constraints
A complex concept set constraint defines two sets of components within a particular concept that must be identical.
Template: T14

### 9.2.5 Valid value definitions
A valid value definition defines the valid values of a particular measure as a range or (occasionally) as a list of discrete values.
Template: T15

### 9.2.6 Data calculation rules
A data calculation rule defines the algorithm or formula for a particular quantity or a conversion factor between two units. They are categorized as:

#### 9.2.6.1 Data calculation algorithms
A data calculation algorithm defines how a particular quantity or amount (whether for operational purposes, such as a fee, or for business intelligence purposes, such as a performance measure) is calculated.
Template: T16

#### 9.2.6.2 Conversion factor definitions
A conversion factor definition defines a conversion factor between two units of measurement.
Template: T17

### 9.2.7 Standard format definitions
A standard format definition defines the standard format for data items of a particular type in terms of individual characters and/or component data items.
Template: T18

## 9.3 Data rules
Data rules (all of which are operative rules) constrains the data included in a transaction (a form or message) or a persistent dataset (e.g., a database record). Data rules can in turn be categorized as:

### 9.3.1 Data cardinality rules
A data cardinality rule requires the presence or absence of a data item and/or places a restriction on the maximum or minimum number of occurrences of a data item

#### 9.3.1.1 Mandatory data rules
A mandatory data rule mandates the presence of data:

##### 9.3.1.1.1 Mandatory data item rules
A mandatory data item rule requires that a particular data item be present.
Template: T19

##### 9.3.1.1.2 Mandatory option selection rules
A mandatory option selection rule requires that one of a set of pre-defined options be specified.
When to apply: Rule statements for rules with two or more options.
Template: T20

When to apply: Rule statements for rules with a single option which may or may not be the case.
Template: T21

##### 9.3.1.1.3 Mandatory group rules:
A mandatory group rule requires that at least one of a group of data items be present.
When to apply: If there are only two data items in the group.
Template: T22

When to apply: If there are more than two data items in the group.
Template: T23

#### 9.3.1.2 Prohibited data rules
A prohibited data rule mandates the absence of some data item in a particular situation.
Template: T24

#### 9.3.1.3 Maximum cardinality rules
A maximum cardinality rule places an upper limit (usually but not necessarily one) on how many instances of a particular data item there may be.
Template: T25

#### 9.3.1.4 Multiple data rules
A multiple data rule mandates the presence of two or more instances of a particular data item in a particular situation.
Template: T19

#### 9.3.1.5 Dependent cardinality rules
A dependent cardinality rule mandates how many of a particular data item must be present based on the value of another data item.
Template: T26

### 9.3.2 Data content rules
A data content rule places a restriction on the values contained in a data item or set of data items (rather than whether they must be present and how many there may or must be).

#### 9.3.2.1 Value set rules
A value set rule requires either: that the content of a data item be (or not be) one of a particular set of values (either a fixed set, or a set that may change over time), or; that the content of a combination of data items match or not match a corresponding combination in a set of records;

##### 9.3.2.1.1 Value set rules constraining single data items
Template: T27

##### 9.3.2.1.2 Value set rules constraining combinations of data items
Template: T28

#### 9.3.2.2. Range rules
A range rule requires that the content of a data item be a value within a particular inclusive or exclusive single-bounded or double-bounded range.
Template: T29

#### 9.3.2.3 Equality rules
An equality rule requires that the content of a data item be the same as or not the same as that of some other data item.
Template: T30

#### 9.3.2.4 Uniqueness constraints
A uniqueness constraint requires that the content of a data item (or combination or set of data items) be different from that of the corresponding data item(s) in the same or other records or transactions;

##### 9.3.2.4.1 Uniqueness constraints constraining single data items.
Template: T31

##### 9.3.2.4.2 Uniqueness constraints constraining combinations of data items
Template: T32

##### 9.3.2.4.3 Uniqueness constraints constraining sets of data items
Template: T33

#### 9.3.2.5 Data consistency rules
A data consistency rule requires the content of multiple data items to be consistent with each other, other than as provided for by a value set rule, range rule, or equality rule;

##### 9.3.2.5.1 Data consistency rules constraining a combination of data items
Template: T34

##### 9.3.2.5.2 Data consistency rules constraining a set function
Template: T35

##### 9.3.2.5.3 Data consistency rules constraining a set
Template: T36

#### 9.3.2.6 Temporal data constraints
A temporal data constraint constrains one or more temporal data items (data items that represent time points or time periods). There are various subcategories of temporal constraint:

##### 9.3.2.6.1 Simple temporal data constraints
A simple temporal data constraint requires that a particular date or time fall within a certain temporal range.
Template: T29

##### 9.3.2.6.2 Temporal data non-overlap constraints
Temporal data non-overlap constraint requires that the time periods specified in a set of records do not overlap each other.
Template: T37

##### 9.3.2.6.3 Temporal data completeness constraints
A temporal data completeness constraint requires that the time periods specified in a set of records be contiguous and between them completely span some other time period.
Template: T38

##### 9.3.2.6.4 Temporal data inclusion constraints
A temporal data inclusion constraint requires that the time periods specified in a set of records do not fall outside some other time period.
Template: T38

##### 9.3.2.6.5 Temporal single record constraints
A temporal single record constraint requires that a temporal state of affairs be recorded using a single record rather than multiple records.
When to apply: a single data item is involved.
Template: T39

When to apply: a combination of data items is involved.
Template: T40

##### 9.3.2.6.6 Day type constraints
A day type constraint restricts a date to one or more days of the week or a particular type of day such as a working day (typically but not necessarily any day other than a Saturday, Sunday, or public holiday).
Template: T41

#### 9.3.2.7 Spatial data constraints
A spatial data constraint prescribes or prohibits relationships between data items representing spatial properties (points, line segments or polygons).
Template: T41

#### 9.3.2.8 Data item format rules
A data item format rule specifies the required format of a data item.
Template: T43

### 9.3.3 Data update rules
A data update rule either prohibits update of a data item or places restrictions on the new value of a data item in terms of its existing value. There are three subcategories of data update rule:

#### 9.3.3.1 Data update prohibition rules
A data update prohibition rule prohibits update of a particular data item or set of data items.
Rule statements for rules governing non-transferable relationships:
Template: T44

Rule statements for other data update prohibition rules:
Template: T45

#### 9.3.3.2 State transition constraints
A state transition constraint limits the changes in a data item to a set of valid transitions.
Template: T46

#### 9.3.3.3 Monotonic transition constraints
A monotonic transition constraint requires that a numeric value either only increase or only decrease.
Template: T47

## 9.4 Activity rules
Activity rules (all of which are operative rules) constrains the operation of one or more business processes or other activities. Activity rules can in turn be categorized as:

### 9.4.1 Activity restriction rules
An activity restriction rule restricts a business process or other activity in some way. There are various subcategories of activity restriction rules:

#### 9.4.1.1 Rules restricting when an activity can occur
Many activity restriction rules place time restrictions on activities.

##### 9.4.1.1.1 Activity time limit rules
An activity time limit rule restricts a business process or other activity to within a particular time period.
Template: T48

##### 9.4.1.1.2 Activity exclusion period rules
An activity exclusion period rule prohibits a business process or other activity during a particular time period.
Template: T49

##### 9.4.1.1.3 Activity obligation rule
An activity obligation rule requires a business process or other activity to occur either within a maximum time after a particular event (such as the completion of some other process) or as soon as practical after a particular event.
Template: T49

#### 9.4.1.2 Activity pre-condition rules
An activity pre-condition rule prohibits a business process or other activity unless some other activity or event has previously occurred or some prerequisite condition exists.
Template: T50

#### 9.4.1.3 Activity prohibition rules
An activity prohibition rule prohibits a business process or other activity if some event or other process has previously occurred or some dangerous or illegal condition exists.
Template: T51

#### 9.4.1.4 Information retention rules
An information retention rule defines the minimum period for which a particular type of information is retained.
Template: T49

#### 9.4.1.5 Activity conflict rules
An activity conflict rule restricts the simultaneous occurrence of multiple processes or other activities.
Template: T?

### 9.4.2 Process decision rules
A process decision rule determines what action a business process or device is to take in specific situations;
9.4.3 Activity obligation rules: An activity obligation rule requires a business process or other activity to occur either within a maximum time after a particular event (such as the completion of some other process) or when particular conditions apply.
Template: T52

## 9.5 Party rules
Party rules (all of which are operative rules) restricts the parties who can perform a process or activity or play a role. Party rules can in turn be categorized as

### 9.5.1 Party restriction rules
A party restriction rule places restrictions on who can perform some processes or activities or play some roles, based on age, some other physical characteristic or capability, or training, testing, and certification in the appropriate skills.
Template: T53

### 9.5.2 Role separation and binding rules
A role separation rule prohibits the same party from performing two activities.
Template: T54

### 9.5.3 Information access rules
An information access rule defines who can view, create, or update particular information.
Template: T55

### 9.5.4 Responsibility rules
A responsibility rule defines who is responsible for performing a particular process or liable for a particular fee, duty, or tax.
Template: T56

Return a JSON object with the following fields.

 If the query is invalid, return an empty classification.

 Here's an example of output:
 [
    {
        "id": "some id",
        "text": "some text",
        "type": "Definitional"
        "subtype": "Formal intensional definitions",
        "templates": ["T7"],
        "examples": ["R70"]
    },
    {
        "id": ...
        "text": ...
        "type": ...
        "subtype": ...
        "templates": ...
        "examples": ...
    },

]

Here's the paragraphs you'll need to classify:
"""

In [ ]:
true_values = [
    {
        "id": "R70",
        "text": "A senior passenger is by definition a passenger whose age is at least 70 years at the time of travel.",
        "type": "Definitional",
        "subtype": "Formal intensional definitions"
    },
    {
        "id": "R73",
        "text": "End of financial year is by definition June 30.",
        "type": "Definitional",
        "subtype": "Symbolic literal definitions"
    },
    {
        "id": "R76",
        "text": "A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment.",
        "type": "Definitional",
        "subtype": "Categorization scheme enumerations"
    },
    {
        "id": "R85",
        "text": "1 ft is by definition equal to 12 in.",
        "type": "Definitional",
        "subtype": "Conversion factor definitions"
    },
    {
        "id": "R91",
        "text": "Each flight booking confirmation must specify exactly one travel class for each flight.",
        "type": "Data",
        "subtype": "Mandatory data item rules"
    },
    {
        "id": "R98",
        "text": "Each flight booking confirmation must specify exactly one of the following: a postal address, an e-mail address, or a fax number.",
        "type": "Data",
        "subtype": "Mandatory group rules"
    },
    {
        "id": "R99",
        "text": "A flight booking request for a one-way journey must not specify a return date.",
        "type": "Data",
        "subtype": "Prohibited data rules"
    },
    {
        "id": "R130",
        "text": "Online check-in for a flight may occur only during the 24 h before the departure time of that flight.",
        "type": "Activity",
        "subtype": "Activity time limit rules"
    },
    {
        "id": "R135",
        "text": "A driver must not operate any vehicle if that driver is intoxicated.",
        "type": "Activity",
        "subtype": "Activity prohibition rules"
    },
    {
        "id": "R140",
        "text": "A person may travel alone only if the age of that person is at least 2 years.",
        "type": "Party",
        "subtype": "Party restriction rules"
    },
    {
        "id": "R78",
        "text": "The status of an employee is by definition one of the following: probational, permanent, or temporary.",
        "type": "Definitional",
        "subtype": "Categorization scheme enumerations"
    },
    {
        "id": "R349",
        "text": "Each flight booking confirmation for an international journey must specify for each passenger specified in that flight booking request a passport number or a visa number but not both.",
        "type": "Data",
        "subtype": "Mandatory group rules"
    },
    {
        "id": "R75",
        "text": "A person is by definition either an adult or a minor.",
        "type": "Definitional",
        "subtype": "Categorization scheme enumerations"
    },
    {
        "id": "R93",
        "text": "Each combination of departure date, flight number, and departure city must be allocated exactly one departure time.",
        "type": "Data",
        "subtype": "Mandatory data item rules"
    },
    {
        "id": "R96",
        "text": "Each flight booking confirmation must specify a mobile phone number, an e-mail address, or both.",
        "type": "Data",
        "subtype": "Mandatory group rules"
    },
    {
        "id": "R143",
        "text": "A person may be rostered on a flight crew only if that person holds an airline transport pilot license that is current and a type endorsement that is current for each aircraft type to be flown by that flight crew.",
        "type": "Party",
        "subtype": "Party restriction rules"
    },
    {
        "id": "R142",
        "text": "A passenger may be allocated to a seat in an exit row only if that passenger is able to open an aircraft door.",
        "type": "Party",
        "subtype": "Party restriction rules"
    },
    {
        "id": "R77",
        "text": "The gender of a person is by definition either male or female.",
        "type": "Definitional",
        "subtype": "Categorization scheme enumerations"
    },
    {
        "id": "R348",
        "text": "Each customer complaint must specify a mobile phone number, an e-mail address, or both.",
        "type": "Data",
        "subtype": "Mandatory group rules"
    }
]

In [ ]:
document1="""
[
    {
        "id": "R70",
        "text": "A senior passenger is by definition a passenger whose age is at least 70 years at the time of travel.",
        "type": "Definitional"
    },
    {
        "id": "R73",
        "text": "End of financial year is by definition June 30.",
        "type": "Definitional"
    },
    {
        "id": "R76",
        "text": "A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment.",
        "type": "Definitional"
    },
    {
        "id": "R85",
        "text": "1 ft is by definition equal to 12 in.",
        "type": "Definitional"
    },
    {
        "id": "R91",
        "text": "Each flight booking confirmation must specify exactly one travel class for each flight.",
        "type": "Data"
    },
    {
        "id": "R98",
        "text": "Each flight booking confirmation must specify exactly one of the following: a postal address, an e-mail address, or a fax number.",
        "type": "Data"
    },
    {
        "id": "R99",
        "text": "A flight booking request for a one-way journey must not specify a return date.",
        "type": "Data"
    },
    {
        "id": "R130",
        "text": "Online check-in for a flight may occur only during the 24 h before the departure time of that flight.",
        "type": "Activity"
    },
    {
        "id": "R135",
        "text": "A driver must not operate any vehicle if that driver is intoxicated.",
        "type": "Activity"
    },
    {
        "id": "R140",
        "text": "A person may travel alone only if the age of that person is at least 2 years.",
        "type": "Party"
    },
    {
        "id": "R78",
        "text": "The status of an employee is by definition one of the following: probational, permanent, or temporary.",
        "type": "Definitional"
    },
    {
        "id": "R349",
        "text": "Each flight booking confirmation for an international journey must specify for each passenger specified in that flight booking request a passport number or a visa number but not both.",
        "type": "Data"
    },
    {
        "id": "R75",
        "text": "A person is by definition either an adult or a minor.",
        "type": "Definitional"
    },
    {
        "id": "R93",
        "text": "Each combination of departure date, flight number, and departure city must be allocated exactly one departure time.",
        "type": "Data"
    },
    {
        "id": "R96",
        "text": "Each flight booking confirmation must specify a mobile phone number, an e-mail address, or both.",
        "type": "Data"
    },
    {
        "id": "R143",
        "text": "A person may be rostered on a flight crew only if that person holds an airline transport pilot license that is current and a type endorsement that is current for each aircraft type to be flown by that flight crew.",
        "type": "Party"
    },
    {
        "id": "R142",
        "text": "A passenger may be allocated to a seat in an exit row only if that passenger is able to open an aircraft door.",
        "type": "Party"
    },
    {
        "id": "R77",
        "text": "The gender of a person is by definition either male or female.",
        "type": "Definitional"
    },
    {
        "id": "R348",
        "text": "Each customer complaint must specify a mobile phone number, an e-mail address, or both.",
        "type": "Data"
    }
]
"""

Query LLM

In [ ]:
response=query_llm(user_prompt=document1, system_prompt=prompt_b, response_model=ParagraphDataset)

In [ ]:
response.paragraphs

#### Validation


Set predictions.

In [ ]:
# Predictions and True values
predictions = response.model_dump()

In [ ]:
labels_predict=[item['subtype'] for item in predictions["paragraphs"]]

In [ ]:
labels_true=[item['subtype'] for item in true_values]

In [ ]:
labels = list(set(labels_true + labels_predict))

In [ ]:
labels

Convert to dataframe.

In [ ]:
# Convert to DataFrames
pred_df = pd.DataFrame(predictions["paragraphs"])
true_df = pd.DataFrame(true_values)

# Merge the two datasets on paragraph_id
merged_df = pd.merge(pred_df, true_df, on="id")

In [ ]:
merged_df

Compute confusion matrix and precision, recall, and accuracy.

In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(merged_df['subtype_y'], merged_df['subtype_x'], labels=labels)

# Compute precision, recall, and accuracy
precision = precision_score(merged_df['subtype_y'], merged_df['subtype_x'], average='weighted')
recall = recall_score(merged_df['subtype_y'], merged_df['subtype_x'], average='weighted')
accuracy = accuracy_score(merged_df['subtype_y'], merged_df['subtype_x'])
f1 = f1_score(merged_df['subtype_y'], merged_df['subtype_x'], average='weighted')

disp = ConfusionMatrixDisplay.from_predictions(merged_df['subtype_y'], merged_df['subtype_x'], labels=labels, xticks_rotation='vertical')

print(f"precison: {precision}, recall: {recall}, accuracy: {accuracy}, f1_score: {f1}")

# Classification evaluation

## Classify (P6)

Two prompts were tested.

As shown in the next sections, the accuracy of prompt a was 0.8947.

The confusion matrix for prompt a reveals misclassifications involving Activity and Party. The table presents the paragraph ID, prediction, and true value. For instance, paragraph R99, which is a Data rule, was misclassified as an Activity. Similarly, paragraph R135, which is an Activity rule, was misclassified as a Party.

In contrast, prompt b achieved 100% correct classifications. The key difference between prompt a and prompt b is that prompt b includes only the definitions of the rules, without instructions on how to classify them.

### prompt a

precison: 0.9078947368421053, recall: 0.8947368421052632, accuracy: 0.8947368421052632

![classify_prompt_a.png](https://github.com/asantos2000/master-degree-santos-anderson/blob/main/code/media/classify_prompt_a.png?raw=1)

> Rows where the prediction differs from the true value are highlighted in **bold**.

| Paragraph ID | Prediction    | Paragraph                                                                                                                                                                    | True Value    |
|--------------|---------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|
| R70          | Definitional  | A senior passenger is by definition a passenger whose age is at least 70 years at the time of travel.                                                                        | Definitional  |
| R73          | Definitional  | End of financial year is by definition June 30.                                                                                                                              | Definitional  |
| R76          | Definitional  | A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment.                                             | Definitional  |
| R85          | Definitional  | 1 ft is by definition equal to 12 in.                                                                                                                                        | Definitional  |
| R91          | Data          | Each flight booking confirmation must specify exactly one travel class for each flight.                                                                                      | Data          |
| R98          | Data          | Each flight booking confirmation must specify exactly one of the following: a postal address, an e-mail address, or a fax number.                                            | Data          |
| **R99**      | **Activity**  | **A flight booking request for a one-way journey must not specify a return date.**                                                                                               | **Data**      |
| R130         | Activity      | Online check-in for a flight may occur only during the 24 h before the departure time of that flight.                                                                        | Activity      |
| **R135**         | **Party**      | **A driver must not operate any vehicle if that driver is intoxicated.**                                                                                                         | **Activity**      |
| R140     | Party  | A person may travel alone only if the age of that person is at least 2 years.                                                                                                | Party      |
| R78          | Definitional  | The status of an employee is by definition one of the following: probational, permanent, or temporary.                                                                       | Definitional  |
| R349         | Data          | Each flight booking confirmation for an international journey must specify for each passenger specified in that flight booking request a passport number or a visa number but not both. | Data          |
| R75          | Definitional  | A person is by definition either an adult or a minor.                                                                                                                        | Definitional  |
| R93          | Data          | Each combination of departure date, flight number, and departure city must be allocated exactly one departure time.                                                          | Data          |
| R96          | Data          | Each flight booking confirmation must specify a mobile phone number, an e-mail address, or both.                                                                             | Data          |
| R143         | Party         | A person may be rostered on a flight crew only if that person holds an airline transport pilot license that is current and a type endorsement that is current for each aircraft type to be flown by that flight crew. | Party         |
| R142         | Party         | A passenger may be allocated to a seat in an exit row only if that passenger is able to open an aircraft door.                                                               | Party         |
| R77          | Definitional  | The gender of a person is by definition either male or female.                                                                                                               | Definitional  |
| R348         | Data          | Each customer complaint must specify a mobile phone number, an e-mail address, or both.                                                                                      | Data          |

### prompt b
precison: 1.0, recall: 1.0, accuracy: 1.0

![image.png](https://github.com/asantos2000/master-degree-santos-anderson/blob/main/code/media/classify_prompt_b.png?raw=1)

> Rows where the prediction differs from the true value are highlighted in **bold**.

| Paragraph ID | Prediction    | Paragraph                                                                                                                                                                    | True Value    |
|--------------|---------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|
| R70          | Definitional  | A senior passenger is by definition a passenger whose age is at least 70 years at the time of travel.                                                                        | Definitional  |
| R73          | Definitional  | End of financial year is by definition June 30.                                                                                                                              | Definitional  |
| R76          | Definitional  | A payment is by definition one of the following: a cash payment, a credit card payment, or an electronic funds transfer payment.                                             | Definitional  |
| R85          | Definitional  | 1 ft is by definition equal to 12 in.                                                                                                                                        | Definitional  |
| R91          | Data          | Each flight booking confirmation must specify exactly one travel class for each flight.                                                                                      | Data          |
| R98          | Data          | Each flight booking confirmation must specify exactly one of the following: a postal address, an e-mail address, or a fax number.                                            | Data          |
| R99      | Activity  | A flight booking request for a one-way journey must not specify a return date.                                                                                               | Activity      |
| R130         | Activity      | Online check-in for a flight may occur only during the 24 h before the departure time of that flight.                                                                        | Activity      |
| R135         | Activity      | A driver must not operate any vehicle if that driver is intoxicated.                                                                                                         | Activity      |
| R140     | Activity  | A person may travel alone only if the age of that person is at least 2 years.                                                                                                | Activity      |
| R78          | Definitional  | The status of an employee is by definition one of the following: probational, permanent, or temporary.                                                                       | Definitional  |
| R349         | Data          | Each flight booking confirmation for an international journey must specify for each passenger specified in that flight booking request a passport number or a visa number but not both. | Data          |
| R75          | Definitional  | A person is by definition either an adult or a minor.                                                                                                                        | Definitional  |
| R93          | Data          | Each combination of departure date, flight number, and departure city must be allocated exactly one departure time.                                                          | Data          |
| R96          | Data          | Each flight booking confirmation must specify a mobile phone number, an e-mail address, or both.                                                                             | Data          |
| R143         | Party         | A person may be rostered on a flight crew only if that person holds an airline transport pilot license that is current and a type endorsement that is current for each aircraft type to be flown by that flight crew. | Party         |
| R142         | Party         | A passenger may be allocated to a seat in an exit row only if that passenger is able to open an aircraft door.                                                               | Party         |
| R77          | Definitional  | The gender of a person is by definition either male or female.                                                                                                               | Definitional  |
| R348         | Data          | Each customer complaint must specify a mobile phone number, an e-mail address, or both.                                                                                      | Data          |

